In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import sys
import pandas as pd
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/features')
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/data')
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from load_data import internal_load
from columnsSpliter import get_columns
import math
import requests
import json
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from zlib import crc32

from sklearn.base import BaseEstimator, TransformerMixin

get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import sys
import pandas as pd
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/features')
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/data')
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from load_data import internal_load
from columnsSpliter import get_columns
import math
import requests
import json
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from zlib import crc32
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/interim/htt_with_lat_long.csv")
original_data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/external/train.csv")

ModuleNotFoundError: No module named 'load_data'

In [9]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)

In [22]:
class create_test_case(BaseEstimator,TransformerMixin):
    def __init__(self, test_ratio, id_column):
        self.test_ratio = test_ratio
        self.id_column = id_column
        return
    def fit(self,data):
        return self
    def transform(self,data,y=None):
        def test_set_check(identifier, test_ratio):
            test_ratio = self.test_ratio
            return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
        def split_train_test_by_id(data):
            ids = data[self.id_column]
            in_test_set = ids.apply(lambda id_: test_set_check(id_, self.test_ratio))
            return data.loc[~in_test_set].reset_index(drop=True)
        return split_train_test_by_id(data)

In [3]:
class address_to_latLong(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self,data):
        return self
    def transform(self,data,y=None):
            switcher = {"Blmngtn":"Bloomington Heights","Blueste":"Bluestem","BrDale":"Briardale","BrkSide":"Brookside","ClearCr":"Clear Creek","CollgCr":"College Creek","Crawfor":"Crawford","Edwards":"Edwards","Gilbert":"Gilbert","IDOTRR":"Iowa DOT and Rail Road","MeadowV":"Meadow Village","Mitchel":"Mitchell","Names":"North Ames","NoRidge":"Northridge","NPkVill":"Northpark Villa","NridgHt":"Northridge Heights","NWAmes":"Northwest Ames","OldTown":"Old Town","SWISU":"South & West of Iowa State University","Sawyer":"Sawyer","SawyerW":"Sawyer West","Somerst":"Somerset","StoneBr":"Stone Brook","Timber":"Timberland","Veenker":"Veenker"}
            def add_fullNeighborhood_columns(data,Neighborhood_attrib):
                new_data = data.copy()
            def Neighborhood_to_fullNeighborhood(argument):
                new_data["fullNeighborhood"] = new_data[Neighborhood_attrib].copy()
                return switcher.get(argument, "nothing")
                for item in new_data[Neighborhood_attrib].index:
                    new_data["fullNeighborhood"][item] = Neighborhood_to_fullNeighborhood(new_data[Neighborhood_attrib][item])
                return new_data
            def get_lat_long(data,address_attrib):
                new_data = data.copy()
                neighborhood_dict = switcher.copy()
                lat_long_dict = neighborhood_dict.copy()
                for key in neighborhood_dict.keys():
                    value = neighborhood_dict.get(key)
                    lat_long_dict[key] = {address_attrib: value, "lat" : 1.1, "long" : 1.1}
                fak = "forward?access_key=a86d264896b9d26e816f31538a0c68a8&query="
                api = "http://api.positionstack.com/v1/"
                for key in lat_long_dict.keys():
                    neighborhood = lat_long_dict.get(key).get(address_attrib)
                    lat = json.loads(requests.get(os.path.join(api,fak,neighborhood)).content)['data'][0]['latitude']
                    long = json.loads(requests.get(os.path.join(api,fak,neighborhood)).content)['data'][0]['longitude']
                    lat_long_dict.get(key)["lat"] = lat
                    lat_long_dict.get(key)["long"] = long
            #         print(key,lat,"/",long)
                new_data["lat"] = new_data["SalePrice"]
                new_data["long"] = new_data["SalePrice"]
                for i in new_data.index:
                    neighborhood = new_data[address_attrib][i]
                    if neighborhood == "NAmes":
                        neighborhood = "Names"
                    lat = lat_long_dict.get(neighborhood).get("lat")
                    long = lat_long_dict.get(neighborhood).get("long")
                    new_data["lat"][i] = lat
                    new_data["long"][i] = long
            #         print (lat, long, neighborhood)
                return new_data
            return get_lat_long(data,'Neighborhood')

In [4]:
class cleaner (BaseEstimator, TransformerMixin):
    def __init__(self, strCols, catCols, numCols, nmcCols):
#         self.string_columns = string_columns
#         self.cat_columns = cat_columns
#         self.numerical_columns = numerical_columns
#         self.numcat_columns = numcat_columns
        self.strCols = strCols
        self.nmcCols = nmcCols
        self.catCols = catCols
        self.numCols = numCols
    def fit(self, data):
        return self
    def transform (self, data,y=None):
        def remove_duplicate (data):
#             print('remove_duplicate')
            data_copy = data.copy()
            data_new = data_copy.drop_duplicates()
            return(data_new.reset_index(drop=True))
        def decapitalizing(data,string_columns,categorical_attribs):
#             print('decapitalizing')
            data_copy = data.copy()
            for column in string_columns: # decapitalizing the data
                data_copy[column] = data_copy[column].str.lower().copy()

            for col in string_columns: # decapitalizing the metadata
                ca_copy = categorical_attribs[col].copy()
                categorical_attribs[col] = [x.lower() for x in ca_copy]
            return data_copy

        def correct_current_typos(data,strCols):
#             print('correct_current_typos')
            data_copy = data.copy()
            data_copy.replace('twnhs','twnhsi',inplace = True)
            data_copy.replace('brk cmn','brkcomm',inplace = True)
            data_copy.replace('cmentbd','cemntbd',inplace = True)
            data_copy.replace('wd shng','wdshing',inplace = True)

            for column in strCols:
                data_copy[column].str.strip()

            return data_copy

        def strc_errors_handler(data,cat_columns, correction = True):
#             print('strc_errors_handler')
            data_copy = data.copy()

            err_cols = {}
            for column in cat_columns:
                i = 0
                while i < len(data_copy[column]):
                    case = data_copy.at[i,column]
                    if case not in cat_columns[column] and not pd.isna(case):
                        print(case,cat_columns[column])
                        if column not in err_cols.keys():
                            err_cols[column] = set()
                        err_cols[column].add(case)
                        if correction:
                            data_copy.drop(i,inplace = True)
                    i += 1
                data_copy = data_copy.reset_index(drop=True)
            return data_copy.reset_index(drop=True)

        def na_handler(data,numerical_columns, numcat_columns,string_columns):
#             print('na_handler')
            data_copy_num = data[numerical_columns].copy()
            data_copy_numcat = data[numcat_columns].copy()
            data_copy_string = data[string_columns].copy()
            data_copy_selected_columns = pd.concat([data_copy_num,data_copy_numcat,data_copy_string],axis=1)
            data_copy_remained = data_copy.drop(columns=data_copy_selected_columns)

            data_copy_num.interpolate(inplace = True)
            for col in data_copy_num:
                i = 0
                if (pd.isna(data_copy_num.at[0,col])):
                    while pd.isna(data_copy_num.at[i,col]):
                        i += 1
                    for j in range(i):
                        data_copy_num.at[j,col] = data_copy_num.at[j+1,col].copy()
                data_copy_numcat.interpolate(method = 'pad', inplace = True)
            data_copy_string.fillna(value = 'missing',inplace = True)
            return pd.concat([data_copy_num,data_copy_numcat,data_copy_string,data_copy_remained], axis=1)
        data_copy = data.copy()
        m1 = remove_duplicate(data_copy).copy().reset_index(drop=True)
        m2 = decapitalizing(m1,self.strCols,self.catCols).copy().reset_index(drop=True)
        m3 = correct_current_typos(m2,self.strCols).copy().reset_index(drop=True)
        m4 = na_handler(m3,self.numCols,self.nmcCols,self.strCols).copy().reset_index(drop=True)
        m5 = strc_errors_handler(m4,self.catCols).copy().reset_index(drop=True)
        return m5

In [5]:
original_data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/external/train.csv")
ds_spliter = create_test_case(0.2,'Id')
train_data,test_data = ds_spliter.fit_transform(original_data)

In [6]:
atl = address_to_latLong()
alted_data = atl.fit_transform(train_data)

<ipython-input-3-9a4bb86ee561>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["lat"][i] = lat
<ipython-input-3-9a4bb86ee561>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["long"][i] = long


In [7]:
clnr = cleaner(get_columns('str'),get_columns('cat'),get_columns('num'),get_columns('nmc'))
cleaned_data = clnr.fit_transform(alted_data)

In [8]:
class feature_scaler(BaseEstimator, TransformerMixin):
    def __init__  (self,numCols):
        self.numCols = numCols
    def fit(self,data):
        return self
    def transform(self,data, y=None):
        def standardizer(data,numCols):
            from sklearn.preprocessing import StandardScaler
            cleaned_num_data = data.copy()
            remaining_data = pd.DataFrame()
            for column in cleaned_num_data:
                if column not in numCols:
                    cleaned_num_data.drop(columns=column,inplace=True)
                    remaining_data[column] = cleaned_data[column]
            stndr = StandardScaler()
            standard_cleaned_num_data = stndr.fit_transform(cleaned_num_data).copy()
            standard_cleaned_num_data_pd = pd.DataFrame(standard_cleaned_num_data, index=cleaned_num_data.index, columns=cleaned_num_data.columns)
            standard_cleaned_data = pd.concat([remaining_data,standard_cleaned_num_data_pd],axis=1)
            return standard_cleaned_data
        return standardizer(data,self.numCols)

In [10]:
fs = feature_scaler(get_columns('num'))
standard_cleaned_data = fs.fit_transform(cleaned_data)

In [11]:
class cat_data_handler(BaseEstimator, TransformerMixin):
    def __init__(self, strCols, nmcCols):   
        self.strCols = strCols
        self.nmcCols = nmcCols
    def fit(self, data):
        return self
    def transform(self,data,y=None):
        def str_column_encoder(data, str_columns):
            data_copy = data.copy()
            new_data = data_copy.copy()
            _cat_encoder = OneHotEncoder() 
            for col in str_columns:
                new_data = new_data.drop(col, axis = 1) # isolate the non categorical data
            for col in str_columns:
                encoded_col = _cat_encoder.fit_transform(data_copy[col].to_numpy().reshape(-1, 1))
                cols =[col + ": " +str(s) for s in _cat_encoder.categories_[0]]
                encoded_col_df = pd.DataFrame(encoded_col.toarray(), columns = cols)
                new_data = pd.concat([new_data, encoded_col_df], axis=1)
                encoded_col_df = pd.DataFrame({})
            return new_data
        return str_column_encoder(data,self.strCols)

In [12]:
encdr = cat_data_handler(get_columns('str'),get_columns('nmc'))
cat_handeled_data = encdr.fit_transform(standard_cleaned_data)

In [13]:
# class transformer_pipeLine(BaseEstimator, TransformerMixin):
# #     def __init__(self,catCols,strCols,numCols,nmcCols,test_ratio,id_column):
#     def __init__(self):
#         self.catCols = get_columns('cat')
#         self.strCols = get_columns('str')
#         self.numCols = get_columns('num')
#         self.nmcCols = get_columns('nmc')
#         self.test_ratio = 2
#         self.id_column = 'Id'
#     def pipe_transform(self,data):
#         from sklearn.pipeline import Pipeline
#         from sklearn.preprocessing import StandardScaler
#         housing_transform_pipeline = Pipeline([
#             ('create_test_case', create_test_case(self.test_ratio,self.id_column)),
#             ('address_to_latLong', address_to_latLong()),
#             ('cleaner', cleaner(self.strCols, self.catCols, self.numCols, self.nmcCols)),
#             ('feature_scaler', feature_scaler(self.numCols)),
#             ('cat_data_handler', cat_data_handler(self.strCols,self.nmcCols))])
#         return housing_transform_pipeline.fit_transform(data)
    
class transformer_pipeLine(BaseEstimator, TransformerMixin):
#     def __init__(self,catCols,strCols,numCols,nmcCols,test_ratio,id_column):
    def pipe_transform(self,data):
        catCols = get_columns('cat')
        strCols = get_columns('str')
        numCols = get_columns('num')
        nmcCols = get_columns('nmc')
        test_ratio = 2
        id_column = 'Id'
        from sklearn.pipeline import Pipeline
        from sklearn.preprocessing import StandardScaler
        housing_transform_pipeline = Pipeline([
            ('create_test_case', create_test_case(test_ratio,id_column)),
            ('address_to_latLong', address_to_latLong()),
            ('cleaner', cleaner(strCols,catCols, numCols, nmcCols)),
            ('feature_scaler', feature_scaler(numCols)),
            ('cat_data_handler', cat_data_handler(strCols,nmcCols))])
        return housing_transform_pipeline.fit_transform(data)

In [14]:
# Now I should make tramsformer pipline
# put the code in right files and places
# train a model
# search how should I test and validate when data is manipulated. Wether done the Transform on test data and then check the result or reverse the result to original format and then check the accuracy and confidence
# go for find out how to reverse data to original format, right for reporting to customer

In [23]:
data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/external/train.csv")
# cdh = transformer_pipeLine(get_columns('cat'),get_columns('str'),get_columns('num'),get_columns('nmc'),2,'Id')
cdh = transformer_pipeLine()
transformed_data = cdh.pipe_transform(data)

<class 'pandas.core.series.Series'>


KeyError: 0

In [ ]:
# read the document of pipeline